In [1]:
from process_bigraph import Composite, pp
import numpy as np

from bsp import app_registrar
from bsp.processes.simple_membrane_process import SimpleMembraneProcess


test_state_init_fp = '../test-membrane-initial.json'
oblate = "../fixtures/sample_meshes/oblate.ply"
prolate = "../fixtures/sample_meshes/prolate.ply"

membrane_config = {
    'characteristic_time_step': 1,
    'geometry': {
        'type': 'icosphere',
        'parameters': {
            'radius': 1.0,
            'subdivision': 2
        }
    },
    'tension_model': {
        'modulus': 0.1,
        'preferred_area': 12.4866
    },
    'osmotic_model': {
        'preferred_volume': 0.7 * 3.14 * 4 / 3,
        'reservoir_volume': 1,
        'strength': 0.02,
        'volume': 2.9
    },
    'parameters': {
        'bending': {
            'Kbc': 8.22e-05
        },
        # 'damping': 0.01
    },
    'tolerance': 1e-11,
    'console_output': False
}




In [2]:
input_composite_state = {
    'membrane': {
        '_type': 'process',
        'address': 'local:simple-membrane-process',
        'config': membrane_config,
        'inputs': {
            'geometry': ['geometry_store'],
            'velocities': ['velocities_store'],
            'protein_density': ['protein_density_store'],
            'volume': ['volume_store'],
            'preferred_volume': ['preferred_volume_store'],
            'reservoir_volume': ['reservoir_volume_store'],
            'surface_area': ['surface_area_store'],
            'osmotic_strength': ['osmotic_strength_store'],
            'duration': ['duration_store']
        },
        'outputs': {
            'geometry': ['geometry_store'],
            'velocities': ['velocities_store'],
            'protein_density': ['protein_density_store'],
            'volume': ['volume_store'],
            'preferred_volume': ['preferred_volume_store'],
            'reservoir_volume': ['reservoir_volume_store'],
            'surface_area': ['surface_area_store'],
            'net_forces': ['net_forces_store'],
            'duration': ['duration_store']
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'geometry': 'GeometryType',
                'velocities': 'VelocitiesType',
                'protein_density': 'ProteinDensityType',
                'volume': 'float',
                'preferred_volume': 'float',
                'reservoir_volume': 'float',
                'surface_area': 'float',
                'net_forces': 'MechanicalForcesType',
            }
        },
        'inputs': {
            'geometry': ['geometry_store'],
            'velocities': ['velocities_store'],
            'protein_density': ['protein_density_store'],
            'volume': ['volume_store'],
            'preferred_volume': ['preferred_volume_store'],
            'reservoir_volume': ['reservoir_volume_store'],
            'surface_area': ['surface_area_store'],
            'net_forces': ['net_forces_store'],
        }
    }
}

In [3]:
sim = Composite(config={'state': input_composite_state}, core=app_registrar.core)

In [4]:
# sim.state['membrane']['instance'].initial_state()

In [5]:
total_time = 3
sim.run(total_time)

Input duration: 0.0 for iteration: 0
Preferred volume: 2.9306666666666668, preferred area: 9.903645223104807
Duration at interval 1.0: 1.0
Input duration: 1.0 for iteration: 1
Preferred volume: 5.8613333333333335, preferred area: 15.721056845476403
Duration at interval 1.0: 1.0
Input duration: 2.0 for iteration: 2
Preferred volume: 11.722666666666667, preferred area: 24.955622174561103
Duration at interval 1.0: 1.0


In [6]:
sim_results = sim.gather_results()[('emitter',)]

In [7]:
# is this (time_0, time_1)?
len(sim_results)

4

In [8]:
x_start = np.array(sim_results[0].get('geometry').get('vertices'))

In [9]:
x_end = sim_results[-1].get('geometry').get('vertices')[:162]

In [ ]:
for i, vertex in enumerate(x_end):
    # print(vertex)
    if len(vertex) != 3:
        print(i)

In [ ]:
sim.state['membrane']['instance'].geometry.getVertexMatrix().shape

In [ ]:
sim_results[0].keys()

In [ ]:
import pymem3dg as dg


p = sim.state['membrane']['instance'].parameters

In [ ]:
from bsp.utils.membrane_utils import parse_parameters

params = parse_parameters(p)

In [ ]:
dir(p.osmotic.form)

In [ ]:
p.osmotic.form = 3